In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../../")
from topological_codes import RotatedDecoder
from topological_codes import RepetitionQubit

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit import execute, Aer

def get_noise_model(p_err):

    error_gate1 = pauli_error([("X", p_err / 2), ("Z", p_err / 2), ("I", 1 - p_err)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "id")
    return noise_model

In [4]:
p_err = 0.10
shots = 10000
def get_results(circ):
    results = execute(circ, 
                      Aer.get_backend("aer_simulator"), 
                      noise_model=get_noise_model(p_err),
                      shots=shots,
                     ).result().get_counts()
    return results

In [5]:
d = 3
T = 1

decoder = RotatedDecoder({"d":d,"T":T})

In [6]:
qubit = RepetitionQubit({'d':d})
qubit.reset_z()
qubit.stabilize()
qubit.id_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",0,get_results(qubit.circ))

qubit = RepetitionQubit({'d':d})
qubit.reset_z()
qubit.x()
qubit.stabilize()
qubit.id_data()
qubit.stabilize()
qubit.readout_z()
decoder.set_look_up_table("Z",1,get_results(qubit.circ))

In [7]:
decoder.look_up_table

{'X': {0: {}, 1: {}},
 'Z': {0: {'1 10 00': 21,
   '1 11 00': 19,
   '0 10 00': 457,
   '0 11 00': 451,
   '0 01 00': 31,
   '1 01 00': 432,
   '0 00 00': 8589},
  1: {'1 01 00': 29,
   '1 10 00': 436,
   '0 01 00': 478,
   '1 11 00': 440,
   '0 11 00': 33,
   '0 10 00': 19,
   '1 00 00': 8565}}}

In [8]:
readout_string = "1 01 00"
readout_type = "Z"
print(decoder._run_look_up(readout_string, readout_type))
decoder.correct_readout_look_up_table(readout_string, readout_type)

[0.9370932754880694, 0.06290672451193058]


0